<a href="https://colab.research.google.com/github/jasoromir/Coding_Interviews/blob/main/Google_Coding_Interview_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Coding Interview With A Normal Software Engineer

This exercise is based on the Youtube [video](https://www.youtube.com/watch?v=rw4s4M3hFfs&ab_channel=Cl%C3%A9mentMihailescu) of former Google Engineer **Clément Mihailescu**.

After the introduction of the problem, I stopped the video and solved it on my own.




The introduction would reads as follows:

You are moving to a new location and want to find the appartment where you are going to move. 
You found a nice street and now you want to find the perfect block in that street where you will moving into.
To find the perfect location we value different facilities, such as gym, shops, school, office, or others. Thus, the perfect location for us would be that which minimize the distance we would have to walk to our required facilities.

So an example of the **inputs** of this problem would be: 

Given a list of *blocks*, where each block contains the information of whether a specific facility is present in that block. And a list of *requirements*.
We need to find the block with minimum walking distance to all of our requirements.

In this case, block number 0 would be at a 1block disntace from a gym, a 0 block distance from a School, and a 4blocks distnace from an Office. So it would have a maximum walking distance of 4.
The best block number would be block number 3, with 1 block distance to Gym and Office and 0 block distance to School. Then maximum distnace would be 1.

In [ ]:
# Input example
Block = [
         {
             'Gym': False,
          'School': True,
          'Office': False
          },
         {
             'Gym': True,
          'School': False,
          'Office': False
          },
         {
             'Gym': True,
          'School': True,
          'Office': False
          },
         {
             'Gym': False,
          'School': True,
          'Office': False
          },
         {
             'Gym': False,
          'School': True,
          'Office': True
          }
         
]

Reqs = ['Gym', 'School', 'Office']

Questions that I would like to have asked:


*   What happens if none of the blocks contain a specified facility (i.e. no gym in any block)?

*   What happens if there are more than one block that have the same maximum walking distance?

Since I don't have the answer to that, let's first solve the problem considering that those cases cannot happen, and then include possible solutions.



Let us first visualize the steps to solve this problem.

Our input could be represented by a matrix such:

| i | Gym | School | Office|  
|---|-----|--------|-------|  
 0  |  0  |   1    |   0   |  
 1  |  1  |   0    |   0   |  
 2  |  1  |   1    |   0   |   
 3  |  0  |   1    |   0   |   
 4  |  0  |   1    |   1   |   

 From which we can compute the distances of each block to the nearest facilty:

| i | Gym | School | Office|  Max_D |
|---|-----|--------|-------|--------| 
 0  |  1  |   0    |   4   |  4
 1  |  0  |   1    |   3   |  3
 2  |  0  |   0    |   2   |  2
 3  |  1  |   0    |   1   |  1
 4  |  2  |   0    |   0   |  2

 And return the block that satisfies the minimum walking distance -> **block 3**

Then we can initialize the variables we are going to need

In [ ]:
L = len(Block)       # Number of blocks in our street
unseen_dist = L + 1  # High number to initialize values of facilities we have not seen yet 
distance_matrix = [0]*L #Matrix where we will store the distances to each facility
max_dist = [0]*L     # Vector of max_distances.

To compute our matrix of distances, we will perform 2 passes, one forward and one backwards.

In the forward pass, we will update the distances of the elements we have seen in previous blocks.

| i | Gym | School | Office| 
|---|-----|--------|-------|
 0  |  6  |   0    |   6   |  
 1  |  0  |   1    |   6   |  
 2  |  0  |   0    |   6   |  
 3  |  1  |   0    |   6   |  
 4  |  2  |   0    |   0   | 

 At the end of the first pass, the last block have been compared to all the other blocks. Now we run the search backwards to update the distances in this other direction.

| i | Gym | School | Office| 
|---|-----|--------|-------|
 0  |  1  |   0    |   4   |  
 1  |  0  |   1    |   3   |  
 2  |  0  |   0    |   2   |  
 3  |  1  |   0    |   1   |  
 4  |  2  |   0    |   0   |  

 Also, since the information given in the list *blocks* may contain more facilities that the ones in our requirements, we will only consider those in the latter to compute the distances.

In [ ]:
# Forward pass

# 1) For each block, go through all the facilities.
# 2) If the facility is in our requirement, then:
# 3) If the facility is present in the current block set distance to 0.
# 4) Else If we are in the first block set distance to maximum
# 5) Else set distance to the minimum among "max_distance" or distance of the previosu block + 1

for idx_block, block in enumerate(Block):
  distance_matrix[idx_block] = {}
  for facility, value in block.items():
    if facility in Reqs:
      if value:
        distance_matrix[idx_block][facility] = 0
      elif idx_block == 0:
        distance_matrix[idx_block][facility] = unseen_dist
      else:
        distance_matrix[idx_block][facility] = min(unseen_dist, distance_matrix[idx_block-1][facility]+1)

# Check matrix so far:
distance_matrix



[{'Gym': 100, 'Office': 100, 'School': 0},
 {'Gym': 0, 'Office': 100, 'School': 1},
 {'Gym': 0, 'Office': 100, 'School': 0},
 {'Gym': 1, 'Office': 100, 'School': 0},
 {'Gym': 2, 'Office': 0, 'School': 0}]

In [ ]:
# Backward pass

# 1) Go through all the blocks, starting at the end
# 2) If the facility is in our requirement, then:
# We dont need to check if facility is true or flase as we already did that.
# 3) If we are in the last block we dont need to update anything and can exctract the maximum disntace
# 4) Else update distances to the minimum among "current_distance" or distance of the previosu block + 1
# 5) Exctract the maximum disntace of that block

for idx_block, block in reversed(list(enumerate(Block))):
  for facility, value in block.items():
    if facility in Reqs:
      if idx_block == L-1:
        max_dist[idx_block] = max(max_dist[idx_block], distance_matrix[idx_block][facility])
      else:
        distance_matrix[idx_block][facility] = min(distance_matrix[idx_block][facility], distance_matrix[idx_block+1][facility]+1)
        max_dist[idx_block] = max(max_dist[idx_block], distance_matrix[idx_block][facility])

# Check distance matrix        
print(distance_matrix)

# Check disntace vector
print(max_dist)

# Print solution
print('The minimum distnace is: ' + str(min(max_dist)) + ' for block number: ', max_dist.index(min(max_dist)))


[{'Gym': 1, 'School': 0, 'Office': 4}, {'Gym': 0, 'School': 1, 'Office': 3}, {'Gym': 0, 'School': 0, 'Office': 2}, {'Gym': 1, 'School': 0, 'Office': 1}, {'Gym': 2, 'School': 0, 'Office': 0}]
[4, 3, 2, 1, 2]
The minimum distnace is: 1 for block number:  3


The running time of this solution would be Big-O(n x m), as we have to run 2 times a nested for loop. Where *N* is the length of the block list and *M* the number of facilities.

# SECOND PART

In [ ]:
Log = [
 [True, True, True, False, False],
 [True, True, True, True,  False],
 [True, True, True, True,  True, True, False, False, False],
 [True, False, False, False, False, False],
 [True, True, True, True,  True, True, True, True, True, True,  True, True, False],
 [True, False],
 [True, True, True, True, False, False]
]

In [ ]:
num_decrease = 0
max_duration = 0
prev_performance = 100

In [ ]:

for el in Log:
  L = len(el)
  green = sum(el)
  percentage_green = green/L*100
  # If we are given the length of each element of the list, we can implement a binary search algorithm to speed this step up

  if percentage_green < prev_performance:
    num_decrease += 1
    max_duration = max(max_duration, num_decrease)
  else:
    num_decrease = 0
  prev_performance = percentage_green

max_duration